In [1]:
# Importing libraries
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.compose import ColumnTransformer

In [2]:
# Reading in the data
ames = pd.read_csv('/content/drive/My Drive/AmesHousing.csv')
ames.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [3]:
# Get rid of columns with mostly NaN values
good_cols = ames.isna().sum() < 100
ames = ames.loc[:,good_cols]

# Drop other NAs
ames = ames.dropna()

# **13.2.5 Your turn**

*Practice Activity*

Consider four possible models for predicting house prices:

Using only the size and number of rooms.
Using size, number of rooms, and building type.
Using size and building type, and their interaction.
Using a 5-degree polynomial on size, a 5-degree polynomial on number of rooms, and also building type.
Set up a pipeline for each of these four models.

Then, get predictions on the test set for each of your pipelines, and compute the root mean squared error. Which model performed best?

Note: You should only use the function train_test_split() one time in your code; that is, we should be predicting on the same test set for all three models.

In [13]:
# Creating response and predictor variables
X = ames.drop('SalePrice', axis=1)
y = ames['SalePrice']

# Splitting into train test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Defining categorical columns
categorical_columns = X.select_dtypes(include=['object']).columns.tolist()

# Defining numerical columns
numerical_columns = ['Gr Liv Area', 'TotRms AbvGrd']

# Pipeline 1: Using only the size and number of rooms
pipeline_1 = Pipeline([
    ('preprocessor', ColumnTransformer([
        ('num', StandardScaler(), numerical_columns)
    ], remainder='drop')),
    ('regressor', LinearRegression())
])

# Pipeline 2: Using size, number of rooms, and building type
pipeline_2 = Pipeline([
    ('preprocessor', ColumnTransformer([
        ('num', StandardScaler(), numerical_columns),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)
    ], remainder='drop')),
    ('regressor', LinearRegression())
])

# Pipeline 3: Using size and building type, and their interaction
ct = ColumnTransformer([
    ('num', StandardScaler(), numerical_columns),
    ('cat', OneHotEncoder(handle_unknown='ignore'), ['Bldg Type'])
], remainder='drop')

# Fitting the preprocessor to get feature names
ct.fit(X_train)
encoded_feature_names = ct.named_transformers_['cat'].get_feature_names_out()

# Creating an interaction transformer
# Calculating the correct indices for the interaction terms
interaction_indices = list(range(len(numerical_columns))) + \
                      list(range(len(numerical_columns), len(numerical_columns) + len(encoded_feature_names)))

ct_inter = ColumnTransformer([
    ('interaction', PolynomialFeatures(degree=2, interaction_only=True, include_bias=False), interaction_indices)
], remainder='drop').set_output(transform = "pandas")

# Creating the pipeline with the preprocessor and interaction transformer
pipeline_3 = Pipeline([
    ('preprocessor', ct),
    ('interaction', ct_inter),
    ('regressor', LinearRegression())
])

# Pipeline 4: Using a 5-degree polynomial on size, a 5-degree polynomial on number of rooms, and also building type
pipeline_4 = Pipeline([
    ('preprocessor', ColumnTransformer([
        ('poly_num', PolynomialFeatures(degree=5), numerical_columns),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)
    ], remainder='drop')),
    ('regressor', LinearRegression())
])

# Storing pipelines in a dictionary
pipelines = {
    'Model 1': pipeline_1,
    'Model 2': pipeline_2,
    'Model 3': pipeline_3,
    'Model 4': pipeline_4
}

# Creating a dictionary to store RMSE scores
rmse_scores = {}

# Evaluating each pipeline
for name, pipeline in pipelines.items():
    pipeline.fit(X_train, y_train)
    predictions = pipeline.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    rmse_scores[name] = rmse
    print(f'RMSE for {name}: {rmse:.2f}')

RMSE for Model 1: 56643.24
RMSE for Model 2: 28243.69
RMSE for Model 3: 55647.30
RMSE for Model 4: 59762.76


*    Model 2 performed the best with a RMSE of 28243.69

# **13.3.1 cross_val_score**

*Practice Activity*

Once again consider four modeling options for house price:

Using only the size and number of rooms.
Using size, number of rooms, and building type.
Using size and building type, and their interaction.
Using a 5-degree polynomial on size, a 5-degree polynomial on number of rooms, and also building type.
Use cross_val_score with the pipelines you made earlier to find the cross-validated root mean squared error for each model.

Which do you prefer? Does this agree with your conclusion from earlier?

In [14]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

# Defining a function to calculate RMSE
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Creating a scorer that can be used in cross_val_score
rmse_scorer = make_scorer(rmse, greater_is_better=False)

# Creating a dictionary for the pipelines
pipelines = {
    'Model 1': pipeline_1,
    'Model 2': pipeline_2,
    'Model 3': pipeline_3,
    'Model 4': pipeline_4
}

# Using cross_val_score to find the RMSE for each model
cv_rmse_scores = {}
for name, pipeline in pipelines.items():
    scores = cross_val_score(pipeline, X, y, scoring=rmse_scorer, cv=5)
    cv_rmse_scores[name] = -scores.mean()  # Negate to get positive RMSE values
    print(f'Cross-validated RMSE for {name}: {cv_rmse_scores[name]:.2f}')

Cross-validated RMSE for Model 1: 55483.72
Cross-validated RMSE for Model 2: 30497.42
Cross-validated RMSE for Model 3: 53296.80
Cross-validated RMSE for Model 4: 61199.27


*    Model 2 had the lowest cross-validated RMSE of 30497.42. This is consistent with the conclusion from earlier.

# **13.3.3 Your Turn**

*Practice Activity*

Consider one hundred modeling options for house price:

House size, trying degrees 1 through 10
Number of rooms, trying degrees 1 through 10
Building Type
Hint: The dictionary of possible values that you make to give to GridSearchCV will have two elements instead of one.

Q1: Which model performed the best?

Q2: What downsides do you see of trying all possible model options? How might you go about choosing a smaller number of tuning values to try?

In [33]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd

ct_poly = ColumnTransformer(
    [
        ("dummify", OneHotEncoder(sparse_output=False), ["Bldg Type"]),
        ("poly_gr_liv_area", PolynomialFeatures(), ["Gr Liv Area"]),
        ("poly_totrms_abvgrd", PolynomialFeatures(), ["TotRms AbvGrd"])
    ],
    remainder="drop"
)

lr_pipeline_poly = Pipeline(
    [
        ("preprocessing", ct_poly),
        ("linear_regression", LinearRegression())
    ]
)

degrees = {
    'preprocessing__poly_gr_liv_area__degree': np.arange(1, 11),
    'preprocessing__poly_totrms_abvgrd__degree': np.arange(1, 11)
}

gscv = GridSearchCV(lr_pipeline_poly, degrees, cv=5, scoring='r2')

# Fitting the model
gscv_fitted = gscv.fit(X, y)

cv_results = gscv_fitted.cv_results_

# Creating a list that includes both parameters
degree_combinations = [
    (param['preprocessing__poly_gr_liv_area__degree'], param['preprocessing__poly_totrms_abvgrd__degree'])
    for param in cv_results['params']
]

# Creating a DataFrame with the degree combinations and their corresponding scores
results_df = pd.DataFrame(data={
    "degree_gr_liv_area": [combo[0] for combo in degree_combinations],
    "degree_totrms_abvgrd": [combo[1] for combo in degree_combinations],
    "scores": cv_results['mean_test_score']
})

results_df

,degree_gr_liv_area,degree_totrms_abvgrd,scores
0,1,1,0.532475
1,1,2,0.530993
2,1,3,0.535925
3,1,4,0.541856
4,1,5,0.540878
...,...,...,...
95,10,6,-32.745290
96,10,7,-32.745129
97,10,8,-32.745145
98,10,9,-32.745241


*Q1: Which model performed the best?*

In [34]:
best_model = gscv_fitted.best_estimator_
best_score = gscv_fitted.best_score_
best_params = gscv_fitted.best_params_

print(f"The best model has the parameters: {best_params} with a score of: {best_score}")

The best model has the parameters: {'preprocessing__poly_gr_liv_area__degree': 4, 'preprocessing__poly_totrms_abvgrd__degree': 5} with a score of: 0.5593144739615847


In [35]:
best_model

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('dummify',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['Bldg Type']),
                                                 ('poly_gr_liv_area',
                                                  PolynomialFeatures(degree=4),
                                                  ['Gr Liv Area']),
                                                 ('poly_totrms_abvgrd',
                                                  PolynomialFeatures(degree=5),
                                                  ['TotRms AbvGrd'])])),
                ('linear_regression', LinearRegression())])

*Q2: What downsides do you see of trying all possible model options? How might you go about choosing a smaller number of tuning values to try?*

*    There is a computational cost of trying all possible model options which can be potentially expensive as well as time-consuming. There is also a risk of overfitting.
*    By using domain knowledge, the range of certain parameters can be limited because they are unlikely to be useful in real world scenarios. Also, RandomizedSearchCV can be used instead to sapmle a fixed number of combinations from specified distributions.